In [ ]:
import torch
from model import BYOLModel
import yaml
weights_pth = '/home/kkallidromitis/tdown/05_07_20-33/05_07_20-33_resnet50_300.pth.tar'
config = torch.load(weights_pth)['config']
config['data']['n_kmeans'] = 4
config['data']['subset'] = ""
model = BYOLModel(config).to('cuda')


In [ ]:
config['data']

In [ ]:
state_dict = {}
checkpoint = torch.load(weights_pth)['model']
length = len(model.state_dict())
keys=list(checkpoint.keys())[:length]
for name, param in zip(model.state_dict(), list(checkpoint.values())[:length]):
    state_dict[name] = param

model.load_state_dict(state_dict, strict=True)

In [ ]:
model.eval()
random_seed = 15 # or any of your favorite number 
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)

In [ ]:
import matplotlib.pyplot as plt

img_dir = '/home/kkallidromitis/data/imagenet/images/train/n02510455/n02510455_3566.JPEG'
img = plt.imread(img_dir)
plt.imshow(img)

new_img = (img_dir,0)

In [ ]:
import torchvision
from torchvision import datasets,transforms
from data.byol_transform import MultiViewDataInjector, get_transform,SSLMaskDataset
image_dir ='/home/kkallidromitis/data/imagenet/images/train'
mask_file_path = None
mask_file= None
transform1 = get_transform('train')
transform2 = get_transform('train', gb_prob=0.1, solarize_prob=0.2)
transform3 = get_transform('raw')

transform = MultiViewDataInjector([transform1, transform2,transform3],config['data']['over_lap_mask'],
                                  config['data']['slic_segments'])

dataset = SSLMaskDataset(image_dir,mask_file,transform=transform,mask_file_path=mask_file_path,
                         subset=config['data'].get("subset", ""))#,specific="n02510455")

from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
import numpy as np

def dump_img(imgs,category):
    n_imgs = len(imgs)
    fig, axes = plt.subplots(1,n_imgs,figsize=(5*n_imgs, 5))
    #raw, kmeans on 
    fig.tight_layout()
    for idx,img in enumerate(imgs):
        axes[idx].imshow(img)
    plt.show()

delay = 1
for images, masks,diff_transfrom,paths in train_dataloader:
    delay -=1
    if delay==0:
        print(images.shape)
        paths = list(paths)
        #images,masks,diff_transfrom = images.unsqueeze(0),masks.unsqueeze(0),diff_transfrom.unsqueeze(0)
        view1 = images[:, 0, ...].contiguous()
        view2 = images[:, 1, ...].contiguous()
        view_raw = images[:, 2, ...].contiguous()
        slic_labelmap = masks[:,2,1,...].contiguous() # B X 1 X H X W
        with torch.no_grad():
            converted_idx_b,converted_idx = model.do_kmeans(view_raw.to('cuda'),slic_labelmap.to('cuda'),False)
        applied_mask = converted_idx
        mask_target = slic_labelmap
        break

In [ ]:
paths

In [ ]:
resize_raw = transforms.Resize((224,224))
totensor = transforms.ToTensor()

In [ ]:
from PIL import Image
images = []
masks = []
slics = []
for i in range(len(mask_target)):
    print(i)
    img_mask = mask_target[i].detach().cpu()
    applied_masked = applied_mask[i].detach().cpu()
    view_rawer = totensor(resize_raw(Image.open(paths[i]))).permute(1,2,0).detach().cpu()
    #print(img_mask.shape,applied_mask.shape,view_rawer.shape)
    dump_img([view_rawer,img_mask,applied_masked],"Masks")
    images.append(view_rawer)
    slics.append(img_mask)
    masks.append(applied_masked)

In [ ]:
resize = transforms.Resize((224,224),interpolation=transforms.functional.InterpolationMode.NEAREST)
grey = transforms.Grayscale()
from matplotlib.pyplot import figure


def combine_img_mask(images,masks,slics,i,labels=[],style=None):
    views = []
    img = grey(images[i].permute(2,0,1)).squeeze().numpy()
    mask = resize(masks[i].unsqueeze(0)).squeeze().numpy()
    
    if len(labels)>0:
        mask = np.where(mask==labels[0], mask, -1)
    fig, ax = plt.subplots(1,3,figsize=(12,6))
    ax[0].axis('off')
    ax[1].axis('off')
    ax[2].axis('off')
    ax[0].imshow(images[i].numpy())
    ax[1].imshow(slics[i].numpy())
    ax[2].imshow(mask, style, interpolation='none')
    ax[2].imshow(img, 'gray', interpolation='none', alpha=0.6)
    plt.tight_layout()
    plt.show()

In [ ]:
index = 10
labels = np.unique(masks[index])
c=0
for i in labels:
    print(i,c)
    c+=1
    combine_img_mask(images,masks,slics,index,[i],'OrRd')

In [ ]:
img = grey(images[index].permute(2,0,1)).squeeze().numpy()
mask = resize(masks[index].unsqueeze(0)).squeeze().numpy()

masks_bin =[]
for l in labels:
        masks_bin.append(np.where(mask==l, mask, -1))
    
fig, ax = plt.subplots(1,4,figsize=(18,6))

ax[0].axis('off')
ax[1].axis('off')
ax[2].axis('off')
ax[3].axis('off')
ax[0].imshow(images[index].numpy())
ax[1].imshow(masks_bin[0], 'OrRd', interpolation='none')
ax[1].imshow(img, 'gray', interpolation='none', alpha=0.6)
ax[2].imshow(masks_bin[1], 'OrRd', interpolation='none')
ax[2].imshow(img, 'gray', interpolation='none', alpha=0.6)
ax[3].imshow(masks_bin[0], 'OrRd', interpolation='none')
ax[3].imshow(img, 'gray', interpolation='none', alpha=0.6)
plt.tight_layout()

plt.show()

In [ ]:
img = grey(images[index].permute(2,0,1)).squeeze().numpy()
mask = resize(masks[index].unsqueeze(0)).squeeze().numpy()

masks_bin =[]
for l in labels:
        masks_bin.append(np.where(mask==l, mask, -1))
    
fig, ax = plt.subplots(1,3,figsize=(18,6))

ax[0].axis('off')
ax[1].axis('off')
ax[2].axis('off')
ax[0].imshow(masks_bin[2], 'Blues', interpolation='none')
ax[0].imshow(img, 'gray', interpolation='none', alpha=0.6)
ax[1].imshow(masks_bin[0], 'Blues', interpolation='none')
ax[1].imshow(img, 'gray', interpolation='none', alpha=0.6)
ax[2].imshow(masks_bin[1], 'Blues', interpolation='none')
ax[2].imshow(img, 'gray', interpolation='none', alpha=0.6)
plt.tight_layout()

plt.show()

In [ ]:
combine_img_mask(images,masks,slics,index,[12],'OrRd')

In [ ]:
for c in ['Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'viridis', 'viridis_r', 'winter', 'winter_r']:
    combine_img_mask(images,masks,slics,0,[42],c)

In [ ]:
img = grey(images[i].permute(2,0,1)).squeeze().numpy()
mask = resize(masks[i].unsqueeze(0)).squeeze().numpy()

In [ ]:
i=218
plt.rcParams["figure.figsize"] = (8,8)
plt.imshow(images[i].numpy())
plt.axis('off')
plt.savefig('panda.png', bbox_inches='tight',pad_inches = 0)

In [ ]:
plt.imshow(np.exp(view1[i].permute(1,2,0)))
plt.axis('off')
plt.savefig('panda_v1.png', bbox_inches='tight',pad_inches = 0)

In [ ]:
plt.imshow(np.exp(view2[i].permute(1,2,0)))
plt.axis('off')
plt.savefig('panda_v2.png', bbox_inches='tight',pad_inches = 0)

In [ ]:
i=218
plt.rcParams["figure.figsize"] = (8,8)
plt.imshow(slics[i].numpy())
plt.axis('off')
plt.savefig('panda_slic.png', bbox_inches='tight',pad_inches = 0)

In [ ]:
img = grey(images[218].permute(2,0,1)).squeeze().numpy()
mask = resize(masks[218].unsqueeze(0)).squeeze().numpy()

plt.imshow(mask, 'jet', interpolation='none')
plt.imshow(img, 'gray', interpolation='none', alpha=0.6)
plt.axis('off')
plt.savefig('panda_reg.png', bbox_inches='tight',pad_inches = 0)

In [ ]:
plt.imshow(mask)
plt.axis('off')
plt.savefig('panda_mask.png', bbox_inches='tight',pad_inches = 0)